In [1]:
import pandas as pd
df = pd.read_csv("../data/fixed_data.csv")
df['day_fixed'] = pd.to_datetime(df['day_fixed'])

In [3]:
import MeCab
import numpy as np
tagger = MeCab.Tagger("-Owakati")

def data_define(d, tokenize=tagger.parse, c1="title", c2="body", sep="。", size=1):
    return tokenize('。'.join([d[1][c1]] + d[1][c2].split(sep)[:size]))

df['data'] = list(map(data_define, df.iterrows()))

In [238]:
import tensorflow as tf
import tensorflow_hub as hub

def texts_encoder(texts):
    with tf.Graph().as_default():
        embed = hub.Module("https://tfhub.dev/google/nnlm-ja-dim128-with-normalization/1")
        embeddings = embed(texts)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            sess.run(tf.tables_initializer())
            result = sess.run(embeddings)
    return result

In [66]:
df = df.sort_values(by='day_fixed')
df = df[df['day_fixed'] > "2017-01-01"][df['day_fixed'] < "2017-12-31"]
df.head()

,Unnamed: 0,body,day,title,url,day_fixed,data
11825,11825,イラク首都バグダッド（Baghdad）中心部シネク（Al-Sinek）地区にある市場で12月...,2017年1月1日 7:40,バグダッドの市場で爆発、死者27人に ISが犯行声明,http://www.afpbb.com/articles/-/3112886,2017-01-01 07:40:00,バグダッド の 市場 で 爆発 、 死者 27 人 に IS が 犯行 声明 。 イラク 首...
11828,11828,国連安全保障理事会（UN Security Council）は12月31日、6年に及ぶシリア...,2017年1月1日 9:03,国連安保理、シリア停戦の決議採択 カザフで和平協議開催へ,http://www.afpbb.com/articles/-/3112889,2017-01-01 09:03:00,国連 安保理 、 シリア 停戦 の 決議 採択 カザフ で 和平 協議 開催 へ 。 国連 ...
15576,37467,中東 中東で過激派組織「イラク・シリア・イスラム国（ＩＳＩＳ）」に対する掃討作戦が続く中、米...,2017.01.01 Sun posted at 09:08 JST,上空２１キロから監視、Ｕ２の偵察任務に密着 CNN EXCLUSIVE,https://www.cnn.co.jp/world/35093723.html,2017-01-01 09:08:00,上空 ２ １ キロ から 監視 、 Ｕ ２ の 偵察 任務 に 密着 CNN EXCLUSI...
15573,37464,トルコの最大都市イスタンブールのベシクタシュ地区にあるナイトクラブで１日未明、襲撃が起きた。...,2017.01.01 Sun posted at 09:29 JST,ナイトクラブに襲撃 トルコ・イスタンブール,https://www.cnn.co.jp/world/35094470.html,2017-01-01 09:29:00,ナイトクラブ に 襲撃 トルコ ・ イスタンブール 。 トルコ の 最大 都市 イスタンブー...
11823,11823,ドナルド・トランプ（Donald Trump）次期米大統領は12月31日、ツイッター（Twi...,2017年1月1日 10:02,米大統領選で惨敗した「敵」に新年の皮肉メッセージ、トランプ氏,http://www.afpbb.com/articles/-/3112890,2017-01-01 10:02:00,米 大統領 選 で 惨敗 し た 「 敵 」 に 新年 の 皮肉 メッセージ 、 トランプ ...


In [239]:
word_vectors = texts_encoder(df['data'].tolist())

INFO:tensorflow:Initialize variable module/embeddings/part_0:0 from checkpoint b'/tmp/tfhub_modules/f2587ea6ed3c5c0de24d63525ca949545fc6de9e/variables/variables' with embeddings


In [381]:
from sklearn.metrics.pairwise import cosine_similarity

def similar_event(word_vectors, source_idx, range_idx, threshold=0.85):
    similar_events = []
    source_vector = word_vectors[source_idx]
    for i in range(range_idx[0], range_idx[0]+range_idx[1]):
        if cosine_similarity([source_vector], [word_vectors[i]])[0][0] > threshold:
            similar_events.append(i)
    return similar_events

In [334]:
events = similar_event(word_vectors, 1, (2, 20000))

In [136]:
df.iloc[1]['data']

'国連 安保理 、 シリア 停戦 の 決議 採択 カザフ で 和平 協議 開催 へ 。 国連 安全 保障 理事 会 （ UN Security Council ） は 12 月 31 日 、 6 年 に 及ぶ シリア 内戦 終結 に 向け 、 ロシア と トルコ が 主導 する 和平 協議 の 決議 案 を 全会 一致 で 採択 し た \n'

In [135]:
df.iloc[events].head()

,Unnamed: 0,body,day,title,url,day_fixed,data
10746,10746,国連安全保障理事会（UN Security Council）は2月28日、化学兵器の使用をめ...,2017年3月1日 5:35,シリア化学兵器使用、中ロが制裁案に拒否権 国連安保理,http://www.afpbb.com/articles/-/3119603,2017-03-01 05:35:00,シリア 化学 兵器 使用 、 中 ロ が 制裁 案 に 拒否 権 国連 安保理 。 国連 安...
15263,37154,ニューヨーク 国連安全保障理事会は２月２８日、シリア政府の化学兵器使用に対する制裁決議案の採...,2017.03.01 Wed posted at 09:39 JST,安保理、シリア制裁決議案を否決 ロシアと中国が拒否権行使,https://www.cnn.co.jp/world/35097344.html,2017-03-01 09:39:00,安保理 、 シリア 制裁 決議 案 を 否決 ロシア と 中国 が 拒否 権 行使 。 ニュ...
10072,10072,国連安全保障理事会（UN Security Council）は6日、北朝鮮が今週実施した弾道...,2017年4月7日 4:05,国連安保理、北ミサイル実験を強く非難,http://www.afpbb.com/articles/-/3124241,2017-04-07 04:05:00,国連 安保理 、 北 ミサイル 実験 を 強く 非難 。 国連 安全 保障 理事 会 （ U...
34305,29809,ロシア外務省は７日、米軍によるシリア攻撃について討議するため、国連安全保障理事会（安保理）の...,"April 7, 2017 / 9:38 AM / a year ago",ロシアが安保理緊急会合を要請、米のシリア攻撃は「無分別」と非難,https://jp.reuters.com/article/mideast-crisis-...,2017-04-07 09:38:00,ロシア が 安保理 緊急 会合 を 要請 、 米 の シリア 攻撃 は 「 無分別 」 と ...
34070,29574,北朝鮮の核開発プログラムを巡る緊張が高まるなか、同国の当局者は１３日、国内で取材中の外国人ジ...,"April 12, 2017 / 10:21 PM / a year ago",緊迫する北朝鮮情勢、外国人記者に「重大なイベント」を通知,https://jp.reuters.com/article/northkorea-even...,2017-04-12 22:21:00,緊迫 する 北朝鮮 情勢 、 外国 人 記者 に 「 重大 な イベント 」 を 通知 。 ...


In [339]:
def grouping_events(events, word_vectors, threshold=0.8):
    results = []
    current_group = []
    
    for i, e in enumerate(events):
        current_group.append(events[i])
        if i == len(events)-1:
            results.append(current_group)
            current_group = []
        else:        
            if cosine_similarity([word_vectors[events[i]]], [word_vectors[events[i+1]]]) > threshold:
                current_group.append(events[i+1])
            else:
                results.append(current_group)
                current_group = []

    if current_group:
        results.append(current_group)
        
    return results        

In [335]:
grouped_events = grouping_events(events, word_vectors, threshold=0.87)
grouped_events

[[3745, 3765, 3765],
 [6319, 6347, 6347, 6701, 6701],
 [7279],
 [7841],
 [9950],
 [11487],
 [12337,
  14042,
  14042,
  14113,
  14113,
  16299,
  16299,
  16424,
  16424,
  16465,
  16465,
  16471]]

In [336]:
df.iloc[grouped_events[-1]]

,Unnamed: 0,body,day,title,url,day_fixed,data
8319,8319,米国は国連安全保障理事会（UN Security Council）の理事国に対して、大陸間弾...,2017年7月7日 8:07,北朝鮮への追加制裁求める声明案、ロシアが反対 国連安保理,http://www.afpbb.com/articles/-/3134819,2017-07-07 08:07:00,北朝鮮 へ の 追加 制裁 求める 声明 案 、 ロシア が 反対 国連 安保理 。 米国 ...
29586,25090,北朝鮮への制裁強化を目指す米国は近く、国連安全保障理事会の理事国１５カ国全てと新たな決議案採...,"August 4, 2017 / 12:45 AM / a year ago",米、北朝鮮制裁強化を安保理15ヵ国と近く協議か＝外交筋,https://jp.reuters.com/article/us-north-korea-...,2017-08-04 00:45:00,米 、 北朝鮮 制裁 強化 を 安保理 15 ヵ国 と 近く 協議 か ＝ 外交 筋 。 北...
29586,25090,北朝鮮への制裁強化を目指す米国は近く、国連安全保障理事会の理事国１５カ国全てと新たな決議案採...,"August 4, 2017 / 12:45 AM / a year ago",米、北朝鮮制裁強化を安保理15ヵ国と近く協議か＝外交筋,https://jp.reuters.com/article/us-north-korea-...,2017-08-04 00:45:00,米 、 北朝鮮 制裁 強化 を 安保理 15 ヵ国 と 近く 協議 か ＝ 外交 筋 。 北...
7754,7754,米国は4日、北朝鮮に対する制裁を強化する国連安全保障理事会（UN Security Coun...,2017年8月5日 7:00,米国、北制裁強化の決議案配布 安保理、5日に採決,http://www.afpbb.com/articles/-/3138206,2017-08-05 07:00:00,米国 、 北 制裁 強化 の 決議 案 配布 安保理 、 5 日 に 採決 。 米国 は 4...
7754,7754,米国は4日、北朝鮮に対する制裁を強化する国連安全保障理事会（UN Security Coun...,2017年8月5日 7:00,米国、北制裁強化の決議案配布 安保理、5日に採決,http://www.afpbb.com/articles/-/3138206,2017-08-05 07:00:00,米国 、 北 制裁 強化 の 決議 案 配布 安保理 、 5 日 に 採決 。 米国 は 4...
7056,7056,米国は8日、国連安全保障理事会（UN Security Council）で11日に北朝鮮への...,2017年9月9日 10:35,米、北朝鮮制裁強化決議案の採決を正式要請,http://www.afpbb.com/articles/-/3142286,2017-09-09 10:35:00,米 、 北朝鮮 制裁 強化 決議 案 の 採決 を 正式 要請 。 米国 は 8 日 、 国...
7056,7056,米国は8日、国連安全保障理事会（UN Security Council）で11日に北朝鮮への...,2017年9月9日 10:35,米、北朝鮮制裁強化決議案の採決を正式要請,http://www.afpbb.com/articles/-/3142286,2017-09-09 10:35:00,米 、 北朝鮮 制裁 強化 決議 案 の 採決 を 正式 要請 。 米国 は 8 日 、 国...
28158,23662,国連安全保障理事会は１１日、６回目の核実験を強行した北朝鮮に対する追加制裁決議案を全会一致で...,"September 11, 2017 / 11:02 PM / 10 months ago",国連安保理、北朝鮮への追加制裁採択 米国「戦争望まない」,https://jp.reuters.com/article/un-north-korea-...,2017-09-11 23:02:00,国連 安保理 、 北朝鮮 へ の 追加 制裁 採択 米国 「 戦争 望ま ない 」 。 国連...
28158,23662,国連安全保障理事会は１１日、６回目の核実験を強行した北朝鮮に対する追加制裁決議案を全会一致で...,"September 11, 2017 / 11:02 PM / 10 months ago",国連安保理、北朝鮮への追加制裁採択 米国「戦争望まない」,https://jp.reuters.com/article/un-north-korea-...,2017-09-11 23:02:00,国連 安保理 、 北朝鮮 へ の 追加 制裁 採択 米国 「 戦争 望ま ない 」 。 国連...
7001,7001,（更新、写真追加）国連安全保障理事会（UN Security Council）は11日（日本...,2017年9月12日 8:28,安保理、北朝鮮制裁決議を全会一致で採択 石油輸出に上限,http://www.afpbb.com/articles/-/3142506,2017-09-12 08:28:00,安保理 、 北朝鮮 制裁 決議 を 全会 一致 で 採択 石油 輸出 に 上限 。 （ 更新...


In [328]:
def regrouping(grouped_events, word_vectors, threshold=0.87):
    results = []
    all_events = []
    current_group = []
    for i, ge1 in enumerate(grouped_events):
        if i in all_events:
            continue
        current_group = []
        current_group += ge1
        for j, ge2 in enumerate(grouped_events[i+1:]):
            if i+j+1 in all_events:
                continue
            if cosine_similarity([word_vectors[ge1[0]]], [word_vectors[ge2[0]]]) > threshold:
                current_group += ge2
                all_events += [i, i+j+1]
        results.append(list(set(current_group)))
    return results

In [366]:
all_events = []
event_instances = []
event_chains = []
for i in range(df.shape[0]):
    if i%1000 == 0:
        print("{}".format(i*100/df.shape[0]))
    if i in all_events:
        continue
    after_week = df.iloc[i]['day_fixed'] + pd.DateOffset(14)
    before_week = df.iloc[i]['day_fixed'] - pd.DateOffset(14)
    len_data = df[before_week <= df['day_fixed']][df['day_fixed'] < after_week].shape[0]
    events = similar_event(word_vectors, i, (i, len_data), threshold=0.87)
    grouped_events = grouping_events(events, word_vectors, threshold=0.87)
    event_instances += grouped_events
    event_chains.append(events)
    all_events += events
    all_events = list(set(all_events))

0.0


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


4.277342914581462
8.554685829162924
12.832028743744386
17.10937165832585
21.38671457290731
25.664057487488773
29.941400402070233
34.2187433166517
38.49608623123316
42.77342914581462
47.050772060396085
51.328114974977545
55.605457889559005
59.882800804140466
64.16014371872193
68.4374866333034
72.71482954788486
76.99217246246631
81.26951537704778
85.54685829162923
89.8242012062107
94.10154412079217
98.37888703537362


In [367]:
len(event_instances), len(event_chains)

(1544, 22861)

In [368]:
regrouped_instances = regrouping(event_instances, word_vectors, threshold=0.87)

In [369]:
regrouped_instances2 = regrouping(regrouped_instances, word_vectors, threshold=0.87)

In [370]:
len(regrouped_instances), len(regrouped_instances2)

(1362, 1280)

In [379]:
%load_ext autoreload
%autoreload 2
import evolution

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [380]:
chains, instances = evolution.execute(df, word_vectors)

NameError: name 'compute_instances_and_chains' is not defined

In [371]:
for i, e in enumerate(regrouped_instances2):
    if df.iloc[e].shape[0] > 1:
        print("[event {}]".format(i))
        print(df.iloc[e].sort_values(by="day_fixed")['title'])
        print()

[event 5]
15577    トルコのナイトクラブに「テロ攻撃」 ３５人死亡、４０人負傷
15575       トルコでナイトクラブ襲撃 死者３９人に、襲撃犯は逃走
Name: title, dtype: object

[event 9]
15568    ギリシャ大使の焼死体見つかる、妻の愛人が殺害か ブラジル
15562      トルコ銃乱射 容疑者の指紋や人相判明、８人の身柄拘束
11283     黒崎さん不明事件、仏がチリに容疑者の拘束・引き渡し要請
15415    フィリピン大統領、警察組織の「浄化」命じる 韓国人殺害で
Name: title, dtype: object

[event 10]
11832    泥酔し操縦室で気失っていたパイロットを逮捕 カナダ
15565    機長が操縦室で意識失う、酒酔いの疑いで逮捕 カナダ
Name: title, dtype: object

[event 12]
11830       「ICBM発射実験の最終段階」金正恩氏が新年の演説で
15555         ＩＣＢＭ発射準備は「最終段階」 北朝鮮の金正恩氏
38321    北朝鮮、ＩＣＢＭ発射実験が最終段階と表明 米国が自制求める
Name: title, dtype: object

[event 19]
38340    中国統計局ＰＭＩ、12月は製造業・非製造業とも低下 50台は維持
37142    中国国家統計局の1月製造業ＰＭＩは51.3に低下、非製造業は改善
Name: title, dtype: object

[event 20]
11804                トルコ銃乱射、犯人逃走で大規模捜査
15564             トルコ銃乱射、単独犯か 容疑者の捜索続く
11816             トルコのナイトクラブ襲撃、ISが犯行声明
11784                トルコ銃乱射、逃走容疑者の写真公開
11792    トルコ銃乱射犯、シリアでIS参加か 実戦経験を犯行に利用？
15566     トルコ銃乱射 実行犯の自撮りビデオ公開、外国籍の２人逮捕
15553      トルコ銃乱射 襲撃犯の足取りが一部判明、タクシーで逃走
11782            トルコ銃乱